# This is a jupyter notebook, running Python 3.6. 
We will use this notebook to import GL / TB demo data, perform some reconciliations, and then perform a few audit procedures.

## Upgrade Pandas library to latest version

In [30]:
!pip install pandas -q --upgrade

## Load libraries

In [1]:
import pandas as pd

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

## Location of gl and tb files

In [3]:
tb = 'data/GLT0_0001_GL account master record transaction figures.xlsx'

In [4]:
gl = 'data/BSEG_0001_Accounting Document Segment.xlsx'

## Pull TB data into Dataframe

In [23]:
tb_df = pd.read_excel(tb, sheet_name=0)

### See what the first five records look like

In [24]:
tb_df.head()

,RCLNT (Not found...),RLDNR (Ledger),RRCTY (Record Type),RVERS (Version),BUKRS (Company Code),RYEAR (Fiscal Year),RACCT (Account Number),RBUSA (Business Area),RTCUR (Currency Key),DRCRK (Debit/Credit Indicator),...,KSL07 (Total transactions in the period in group currency (curr 3)),KSL08 (Total transactions in the period in group currency (curr 3)),KSL09 (Total transactions in the period in group currency (curr 3)),KSL10 (Total transactions in the period in group currency (curr 3)),KSL11 (Total transactions in the period in group currency (curr 3)),KSL12 (Total transactions in the period in group currency (curr 3)),KSL13 (Total transactions in the period in group currency (curr 3)),KSL14 (Total transactions in the period in group currency (curr 3)),KSL15 (Total transactions in the period in group currency (curr 3)),KSL16 (Total transactions in the period in group currency (curr 3))
0,800,0,0,1,3000,1994,1000,1000,USD,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,800,0,0,1,3000,1994,1000,7000,USD,H,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
2,800,0,0,1,3000,1994,1000,7000,USD,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,800,0,0,1,3000,1994,1000,9900,USD,S,...,0.00,0.00,0.00,0.00,"2,980,821.41","644,372.42",0.00,0,0,0
4,800,0,0,1,3000,1994,1010,1000,USD,H,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0


## Reshape the data to unpivot periods from columns to rows

In [37]:
tb_column_renames = {
    'RACCT (Account Number)': 'GL_Account_Number',
    'RBUSA (Business Area)': 'Business_Unit_Code',
    'RYEAR (Fiscal Year)': 'Fiscal_Year',
    'RLDNR (Ledger)': 'Ledger'
}

In [38]:
tb_df_melt = tb_df.melt(id_vars=tb_column_renames.keys(),
    value_vars=['TSL01 (Total transactions of the period in transaction currency)',
       'TSL02 (Total transactions of the period in transaction currency)',
       'TSL03 (Total transactions of the period in transaction currency)',
       'TSL04 (Total transactions of the period in transaction currency)',
       'TSL05 (Total transactions of the period in transaction currency)',
       'TSL06 (Total transactions of the period in transaction currency)',
       'TSL07 (Total transactions of the period in transaction currency)',
       'TSL08 (Total transactions of the period in transaction currency)',
       'TSL09 (Total transactions of the period in transaction currency)',
       'TSL10 (Total transactions of the period in transaction currency)',
       'TSL11 (Total transactions of the period in transaction currency)',
       'TSL12 (Total transactions of the period in transaction currency)'],
       var_name='Period', 
       value_name='Balance_As_Of_Date')

## Rename the period fields

In [39]:
tb_df_melt['Period'] = tb_df_melt['Period'].map(lambda x: x[3:5])

In [40]:
tb_df_melt = tb_df_melt.rename(columns=tb_column_renames)

In [41]:
tb_df_melt.head()

,GL_Account_Number,Business_Unit_Code,Fiscal_Year,Ledger,Period,Balance_As_Of_Date
0,1000,1000,1994,0,01,"484,570.00"
1,1000,7000,1994,0,01,"-2,000,000.00"
2,1000,7000,1994,0,01,"2,000,000.00"
3,1000,9900,1994,0,01,0.00
4,1010,1000,1994,0,01,"-1,616.00"


In [42]:
# Save file for import example
tb_df_melt.to_csv('data/Trial_Balance_YYYYMMDD_YYYYMMDD.csv', index=False)

# Pull data from gl into Dataframe

In [46]:
gl_df = pd.read_excel(gl, sheet_name=0)

In [29]:
#gl_df = gl_df[['BELNR (Accounting Document Number)', 'BUZEI (Number of Line Item Within Accounting Document)', 
#               'SGTXT (Item Text)', 'GSBER (Business Area)', 'AUGDT (Clearing Date)','GJAHR (Fiscal Year)',
#              'HKONT (General Ledger Account)', 'PSWBT (Amount for Updating in General Ledger)',
#              'SHKZG (Debit/Credit Indicator)', 'PSWSL (Update Currency for General Ledger Transaction Figures)']]

In [47]:
gl_df.head()

,MANDT (Not found...),BUKRS (Company Code),BELNR (Accounting Document Number),GJAHR (Fiscal Year),BUZEI (Number of Line Item Within Accounting Document),BUZID (Identification of the Line Item),AUGDT (Clearing Date),AUGCP (Clearing Entry Date),AUGBL (Document Number of the Clearing Document),BSCHL (Posting Key),...,PRCTR (Profit Center),XHKOM (Indicator: G/L account assigned manually ?),PROJK (Work Breakdown Structure Element (WBS Element)),DMBE2 (Amount in Second Local Currency),HWMET (Method with Which the Local Currency Amount Was Determined),XRAGL (Indicator: Clearing was Reversed),XNEGP (Indicator: Negative posting),KIDNO (Payment Reference),AUGGJ (Fiscal Year of Clearing Document),SEGMENT (Segment for Segmental Reporting)
0,800,3000,100000000,2007,1,NaN,19000101,19000101,nan,40,...,3402,nan,NaN,"7,574.05",NaN,nan,nan,nan,nan,NaN
1,800,3000,100000000,2007,2,NaN,19000101,19000101,nan,50,...,NaN,nan,NaN,"7,574.05",NaN,nan,nan,nan,nan,NaN
2,800,3000,100000001,2007,1,NaN,19000101,19000101,nan,40,...,3402,nan,NaN,"4,554.42",NaN,nan,nan,nan,nan,NaN
3,800,3000,100000001,2007,2,NaN,19000101,19000101,nan,50,...,NaN,nan,NaN,"4,554.42",NaN,nan,nan,nan,nan,NaN
4,800,3000,100000002,2007,1,NaN,19000101,19000101,nan,40,...,3402,nan,NaN,189.77,NaN,nan,nan,nan,nan,NaN


In [48]:
gl_column_rename_bseg = {
    'BELNR (Accounting Document Number)': 'Journal_ID',
    'BUZEI (Number of Line Item Within Accounting Document)': 'Journal_ID_Line_Number',
    'SGTXT (Item Text)': 'JE_Line_Description',
    'GSBER (Business Area)': 'Business_Unit_Code',
    'AUGDT (Clearing Date)': 'Effective_Date',
    'GJAHR (Fiscal Year)': 'Fiscal_Year',
    'HKONT (General Ledger Account)': 'GL_Account_Number',
    'PSWBT (Amount for Updating in General Ledger)': 'Amount',
    'SHKZG (Debit/Credit Indicator)': 'Amount_Credit_Debit_Indicator',
    'PSWSL (Update Currency for General Ledger Transaction Figures)': 'Amount_Currency'

}

gl_column_rename_bkpf = {
    'BKTXT (Document Header Text)': 'JE_Header_ Description',
    'BLART (Document Type)': 'Source',
    'USNAM (User name)': 'Entered_By',
    'BLDAT (Document Date in Document)' : 'Document_Date', 
    'CPUDT (Day On Which Accounting Document Was Entered)': 'Entered_Date',
    'CPUTM (Time of Entry)': 'Entered_Time',
    'BELNR (Accounting Document Number)': 'Journal_ID',
    'MONAT (Fiscal Period)': 'Period'
}

In [49]:
# gl_df['Net'] = gl_df.apply(lambda x: x['PSWBT (Amount for Updating in General Ledger)']
#                            if x['SHKZG (Debit/Credit Indicator)'] == 'H'
#                           else (x['PSWBT (Amount for Updating in General Ledger)'] * -1),
#                           axis=1)

In [50]:
gl_df = gl_df.rename(columns=gl_column_rename_bseg)

In [51]:
cols = list(gl_column_rename_bseg.values())
gl_df_renamed = gl_df[cols]

In [52]:
gl_df_renamed.head()

,Journal_ID,Journal_ID_ Line_Number,JE_Line_ Description,Business_Unit_ Code,Effective_Date,Fiscal_Year,GL_Account_ Number,Amount,Amount_Credit_ Debit_Indicator,Amount_ Currency
0,100000000,1,Postkosten ohne Tel.,9900,19000101,2007,473000,"9,770.52",S,USD
1,100000000,2,NaN,NaN,19000101,2007,113100,"9,770.52",H,USD
2,100000001,1,Reisekst./Unterkunft,9900,19000101,2007,474210,"5,875.20",S,USD
3,100000001,2,NaN,NaN,19000101,2007,113100,"5,875.20",H,USD
4,100000002,1,NaN,9900,19000101,2007,474211,244.80,S,USD


## Load file to pull other fields from

In [50]:
xwalk = 'data/BKPF_0001_Accounting Document Header.TXT'

In [51]:
xwalk_df = pd.read_csv(xwalk, sep='|', low_memory=False)

In [52]:
xwalk_df = xwalk_df.rename(columns=gl_column_rename_bkpf)

In [53]:
cols = list(gl_column_rename_bkpf.values())

In [54]:
xwalk_final = xwalk_df[cols]

In [55]:
xwalk_final.head()

,JE_Header_Description,Source,Entered_By,Document_Date,Entered_Date,Entered_Time,Journal_ID,Period
0,NaN,SA,STEINER,20070101,20070122,101208,100000004,1
1,NaN,SA,STEINER,20070101,20070122,101207,100000003,1
2,NaN,SA,STEINER,20070101,20070122,101206,100000002,1
3,NaN,SA,STEINER,20070101,20070122,101206,100000001,1
4,NaN,SA,STEINER,20070101,20070122,101205,100000000,1


In [56]:
gl_df_final = pd.merge(gl_df_renamed, xwalk_final, on='Journal_ID', how='left')
gl_df_final.head()

,Journal_ID,Journal_ID_Line_Number,JE_Line_Description,Business_Unit_Code,Effective_Date,Fiscal_Year,GL_Account_Number,Amount,Amount_Credit_Debit_Indicator,Amount_Currency,JE_Header_Description,Source,Entered_By,Document_Date,Entered_Date,Entered_Time,Period
0,100000000,1,Postkosten ohne Tel.,9900,19000101,2007,473000,"9,770.52",S,USD,NaN,SA,STEINER,20070101,20070122,101205,1
1,100000000,2,NaN,NaN,19000101,2007,113100,"9,770.52",H,USD,NaN,SA,STEINER,20070101,20070122,101205,1
2,100000001,1,Reisekst./Unterkunft,9900,19000101,2007,474210,"5,875.20",S,USD,NaN,SA,STEINER,20070101,20070122,101206,1
3,100000001,2,NaN,NaN,19000101,2007,113100,"5,875.20",H,USD,NaN,SA,STEINER,20070101,20070122,101206,1
4,100000002,1,NaN,9900,19000101,2007,474211,244.80,S,USD,NaN,SA,STEINER,20070101,20070122,101206,1


In [57]:
# Save the gl to csv
gl_df_final.to_csv('data/GL_Detail_YYYYMMDD_YYYYMMDD.csv', index=False)